## Model 1.0
Powered by [Eleonora Priori](https://www.est-en.unito.it/do/docenti.pl/Alias?eleonora.priori#tab-profilo) and [Pietro Terna](https://terna.to.it/) 


In [1]:
%%javascript
// to avoid scroll in windows
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

====================================================================================================

## 1

import libs \
MPI init \
context and runner definition \
t(), T(), Tc() function definitions \
random number generator rng creation \
initialization of the parameters from yaml file\
memory allocations to manage ghosts


====================================================================================================

In [2]:
import time
from mpi4py import MPI
from repast4py import context as ctx
import repast4py 
from repast4py import parameters
from repast4py import schedule
from repast4py import core
from typing import Tuple, List, Dict
import numpy as np
import csv


comm = MPI.COMM_WORLD
rank    = comm.Get_rank()
rankNum = comm.Get_size() 

# create the context to hold the agents and manage cross process
# synchronization
context = ctx.SharedContext(comm)

# Initialize the default schedule runner, HERE to create the t() function,
# returning the tick value
runner = schedule.init_schedule_runner(comm)

# tick number
def t():
    return int(runner.schedule.tick)

#Initializes the repast4py.parameters.params dictionary with the model input parameters.
params = parameters.init_params("model1.yaml", "")


#generate random seed
repast4py.random.init(rng_seed=params['myRandom.seed'][rank]) #each rank has a seed
rng = repast4py.random.default_rng 


#timer T()
startTime=-1
def T():
    global startTime
    if startTime < 0:
        startTime=time.time()
    return time.time() - startTime
T()

#cpuTimer Tc()
startCpuTime=-1
def Tc():
    global startCpuTime
    if startCpuTime < 0:
        startCpuTime=time.process_time()
    return time.process_time() - startCpuTime
Tc()

agent_cache={} # dict with uid as keys and agents' tuples as values

===================================================================================================

## 2

create agents' classes and restore_agent function 



===================================================================================================

In [3]:
class Firm(core.Agent):

    TYPE = 0
    
    def __init__(self, local_id: int, rank: int, capital:float, labor:float,\
                 duration:int, recipe: float, laborProductivity: float, dimensionalClass: int):
        super().__init__(id=local_id, type=Firm.TYPE, rank=rank) #uid
        
        self.capital=capital
        self.labor=labor
        self.duration=duration
        self.recipe = recipe
        self.laborProductivity=laborProductivity
        self.dimensionalClass=dimensionalClass
        self.lostProduction=0
        self.inventories=0
        self.inProgressInventories=0
        self.appRepository=[] #aPP=aProductiveProcess
        self.availableLabor=self.labor
        self.availableCapital=self.capital
        
    def produce(self, productionOrder:float)->tuple:
        
        output=0
        costOfProductionOrder=0
        costOfUnusedFactors=0
        totalRequiredCapital=0
        totalRequiredLabor=0
        totalLostProduction=0
        totalCostOfLostProduction=0
        
        
        targetProductionOfThePeriod=productionOrder/self.duration
        requiredLabor=np.ceil(targetProductionOfThePeriod/self.laborProductivity)
        requiredCapital=requiredLabor*self.recipe

        #create a new aPP or skip the order
        if requiredLabor <= self.labor and requiredCapital <= self.capital: 
            aProductiveProcess=ProductiveProcess(targetProductionOfThePeriod, \
                                                 requiredCapital, requiredLabor, self.duration)
            self.appRepository.append(aProductiveProcess)
        
        for aProductiveProcess in self.appRepository:  
            if aProductiveProcess.hasResources or \
            (self.availableCapital >= aProductiveProcess.requiredCapital and \
             self.availableLabor >= aProductiveProcess.requiredLabor):
                
                if not aProductiveProcess.hasResources:
                    self.availableCapital -= aProductiveProcess.requiredCapital
                    self.availableLabor -= aProductiveProcess.requiredLabor
                    aProductiveProcess.hasResources = True 
                #production
                (outputOfThePeriod, requiredCapital, requiredLabor,\
                lostProduction, costOfLostProduction)=aProductiveProcess.step(productionOrder)
                
                     
                output += outputOfThePeriod
                cost = requiredCapital*params['costOfCapital'] + requiredLabor*params['wage']
                
                costOfProductionOrder += cost
                if aProductiveProcess.productionClock < aProductiveProcess.duration:
                    self.inProgressInventories += cost
                else:
                    self.inventories=cost*self.duration
                    self.inProgressInventories -= cost*(self.duration-1)
                    
                
                totalRequiredCapital += requiredCapital
                totalRequiredLabor += requiredLabor
                totalLostProduction += lostProduction
                totalCostOfLostProduction += costOfLostProduction
            
        costOfUnusedFactors = self.availableCapital*params['costOfCapital'] +\
                              self.availableLabor*params['wage']
        
        for aProductiveProcess in self.appRepository:
            if aProductiveProcess.productionClock == aProductiveProcess.duration: 
                self.availableCapital+=aProductiveProcess.requiredCapital
                self.availableLabor+=aProductiveProcess.requiredLabor
                self.appRepository.remove(aProductiveProcess)
                del aProductiveProcess
                
        #costOfUnusedFactors = self.availableCapital*params['costOfCapital'] +\
                              #self.availableLabor*params['wage']
        
        print(self.uid,"\n", productionOrder, self.capital, self.labor, self.recipe, self.laborProductivity,"\n", \
              output, costOfProductionOrder, costOfUnusedFactors, self.inventories,\
              self.inProgressInventories, totalLostProduction,totalCostOfLostProduction, flush=True)
        
        return(output, costOfProductionOrder, costOfUnusedFactors,self.inventories,\
               self.inProgressInventories, totalLostProduction, totalCostOfLostProduction)
        

    
    
class ProductiveProcess():
    def __init__(self, targetProductionOfThePeriod:float, requiredCapital:float, requiredLabor:float, duration:int):
        
        self.targetProductionOfThePeriod=targetProductionOfThePeriod
        self.requiredCapital = requiredCapital
        self.requiredLabor = requiredLabor
        self.duration = duration
        self.productionClock=0
        self.costOfLostProduction=0
        self.hasResources= False
        
    def step(self, productionOrder)->tuple:
        
        lostProduction=0
        self.productionClock += 1
        
        if params['probabilityToFailProductionChoices'] >= rng.random():
            self.targetProductionOfThePeriod=0
            lostProduction=self.targetProductionOfThePeriod*self.productionClock
            self.costOfLostProduction=(params['wage']* self.requiredLabor+\
                                       params['costOfCapital']* self.requiredCapital)*self.productionClock
            self.productionClock = self.duration   
        return(self.targetProductionOfThePeriod, self.requiredCapital, self.requiredLabor, \
               lostProduction, self.costOfLostProduction)

===================================================================================================

## 3

the model

===================================================================================================

In [4]:
class Model:
    
    global params
    PARAMS = params
    
    def __init__(self, params: Dict):
        
        self.totalProduction=[]
        self.totalCostOfProduction=[]
        self.totalCostOfUnusedFactors=[]
        self.totalInventories=[]
        self.totalInProgressInventories=[]
        self.totalLostProduction=[]
        self.totalCostOfLostProduction=[]
        
        #the context and the runner are created in step 1 
      
        runner.schedule_event(          0.0,     self.initGhosts) 
        runner.schedule_repeating_event(0.0,  1, self.counter)
        runner.schedule_repeating_event(0.1,  1, self.firmsProducing)
        
        runner.schedule_stop(params['howManyCycles'])
        runner.schedule_end_event(self.finish)
        
        ####################################################################################################
        ###################################### CREATE FIRM AGENTS ##########################################
        ####################################################################################################
        
        #importing csv file containing info about firms 
        #share of firms of that class, L-min, L-max, K-min, K-max, t-min, t-max, recipe, L-prod
        with open('firm-features.csv', newline='') as csvfile:
            firmReader= csv.reader(csvfile, delimiter=',', quoting=csv.QUOTE_NONNUMERIC)
            
            self.rowNumber=0
            k=0
            for row in firmReader:
                print(row)
                for i in range(int(row[0] * params['Firm.count'])// rankNum):
                    labor= rng.integers(row[1], row[2]+1)
                    capital= row[3] + rng.random()*(row[4] -row[3])
                    duration= rng.integers(row[5], row[6]+1)
                    recipe= row[7] #L/K 
                    laborProductivity= row[8]
                    dimensionalClass=self.rowNumber
                    aFirm =Firm(k, rank, capital, labor, duration, recipe, laborProductivity, dimensionalClass)
                    context.add(aFirm)
                    k += 1
                self.rowNumber += 1

        
    #initialize ghosts by sending them in the ranks before starting the simulation
    def initGhosts(self):
        pass
    
    #count the cycles number
    def counter(self):
        if int(t()) % params["tickNumber.betweenChecks"] == 0: 
            print("rank", rank, "tick", t(), flush=True)
            
  
    
    def firmsProducing(self):
        self.totalProduction.append([0]*(self.rowNumber)) #for each cycle adds a sub-list of lenght number of dimClass
        self.totalCostOfProduction.append([0]*(self.rowNumber))
        self.totalCostOfUnusedFactors.append([0]*(self.rowNumber))
        self.totalInventories.append([0]*(self.rowNumber))
        self.totalInProgressInventories.append([0]*(self.rowNumber))
        self.totalLostProduction.append([0]*(self.rowNumber))
        self.totalCostOfLostProduction.append([0]*(self.rowNumber))
        
        for aFirm in context.agents(agent_type=0):
            #print(aFirm.dimensionalClass)
            tupleOfProductionResults = aFirm.produce(rng.random()*params['maxOrderProduction'])
            self.totalProduction[t()][aFirm.dimensionalClass] += tupleOfProductionResults[0]
            self.totalCostOfProduction[t()][aFirm.dimensionalClass] += tupleOfProductionResults[1]
            self.totalCostOfUnusedFactors[t()][aFirm.dimensionalClass] += tupleOfProductionResults[2]
            self.totalInventories[t()][aFirm.dimensionalClass] += tupleOfProductionResults[3]
            self.totalInProgressInventories[t()][aFirm.dimensionalClass] += tupleOfProductionResults[4] 
            self.totalLostProduction[t()][aFirm.dimensionalClass] += tupleOfProductionResults[5]
            self.totalCostOfLostProduction[t()][aFirm.dimensionalClass] += tupleOfProductionResults[6]
    
            
    #finish
    def finish(self):
        
        # infos for data_analysis.ipynb
        with open('plotInfo.csv', 'w', newline='')\
          as file:
            writer = csv.writer(file)
            writer.writerow((params["log_file_root"],rankNum))
        
        
        print("\n total production", self.totalProduction, flush=True)
        print("\n cost of production", self.totalCostOfProduction, flush=True)
        print("\n total cost of unused factors", self.totalCostOfUnusedFactors, flush=True)
        print("\n total lost production", self.totalLostProduction, flush=True)

        print("THE END!", flush=True)
        
        names=["_production_","_cost of production_","_total cost of unused factors_",
              "_total lost production_"]
        contents=[self.totalProduction,self.totalCostOfProduction,\
                  self.totalCostOfUnusedFactors,self.totalLostProduction]
        
        for s in range(4):
            with open(params["log_file_root"]+names[s]+str(rank)+'.csv', 'w',\
                  newline='') as file:
                writer = csv.writer(file)
                for k in range(params["howManyCycles"]):
                    writer.writerow(contents[s][k])
    
    def start(self):
        runner.execute()

=========================================================================================================

## 4

run the model

==========================================================================================================

In [5]:
def run(params: Dict):
    
    model = Model(params) 
    model.start()
    
run(params)

[0.1, 80.0, 100.0, 250.0, 600.0, 1.0, 1.0, 5.0, 0.8]
[0.3, 40.0, 60.0, 50.0, 300.0, 1.0, 1.0, 2.0, 0.7]
[0.6, 20.0, 40.0, 5.0, 40.0, 1.0, 1.0, 0.5, 0.6]
rank 0 tick 0
(0, 0, 0) 
 11.697809232306614 532.9467523536742 100 5.0 0.8 
 11.697809232306614 22.5 130.79467523536744 22.5 0.0 0 0
(1, 0, 0) 
 17.423036210841186 183.25296813496047 40 2.0 0.7 
 17.423036210841186 30.0 28.325296813496045 30.0 0.0 0 0
(2, 0, 0) 
 8.01449513167607 72.98299234231591 45 2.0 0.7 
 0 14.4 37.898299234231594 14.4 0.0 0 14.4
(3, 0, 0) 
 12.952796072516495 268.61165593376273 42 2.0 0.7 
 12.952796072516495 22.8 46.06116559337627 22.8 0.0 0 0
(4, 0, 0) 
 7.902826688586522 35.6571463228715 35 0.5 0.6 
 7.902826688586522 14.7 23.86571463228715 14.7 0.0 0 0
(5, 0, 0) 
 5.561570837400909 22.690968920329617 21 0.5 0.6 
 5.561570837400909 10.5 12.769096892032962 10.5 0.0 0 0
(6, 0, 0) 
 17.34524975136447 20.559264474106044 29 0.5 0.6 
 17.34524975136447 30.45 0.6059264474106044 30.45 0.0 0 0
(7, 0, 0) 
 6.52149884880

 15.419873039399427 27.3 12.195044170994258 27.3 0.0 0 0
rank 0 tick 7
(0, 0, 0) 
 12.824223542824228 532.9467523536742 100 5.0 0.8 
 12.824223542824228 25.5 127.79467523536744 25.5 0.0 0 0
(1, 0, 0) 
 2.0835727157197415 183.25296813496047 40 2.0 0.7 
 2.0835727157197415 3.6 54.725296813496044 3.6 0.0 0 0
(2, 0, 0) 
 16.2204396313691 72.98299234231591 45 2.0 0.7 
 16.2204396313691 28.8 23.49829923423159 28.8 0.0 0 0
(3, 0, 0) 
 6.290483310632884 268.61165593376273 42 2.0 0.7 
 0 10.8 58.06116559337627 10.8 0.0 0 10.8
(4, 0, 0) 
 14.861573464227371 35.6571463228715 35 0.5 0.6 
 14.861573464227371 26.25 12.31571463228715 26.25 0.0 0 0
(5, 0, 0) 
 10.064315840226463 22.690968920329617 21 0.5 0.6 
 10.064315840226463 17.85 5.419096892032962 17.85 0.0 0 0
(6, 0, 0) 
 16.94356758249134 20.559264474106044 29 0.5 0.6 
 16.94356758249134 30.45 0.6059264474106044 30.45 0.0 0 0
(7, 0, 0) 
 9.197149505025212 19.470629611983405 25 0.5 0.6 
 9.197149505025212 16.8 10.14706296119834 16.8 0.0 0 0
(8, 